####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 5.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-19 10:49:42--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-19 10:49:42--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

#### Gradient Boosting Next Day 
***

#####Load Dataset
***

In [2]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   #change fb to meta 25/08/2022
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-08-22')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο 
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-08-22')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd1= rd.pct_change().shift(-1)
rd1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,-0.007219,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002198,...,-0.004424,0.011185,-0.010431,0.004739,-0.016101,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-02,0.007271,0.003619,0.013997,-0.002892,-0.018919,0.004302,-0.014527,0.001645,-0.000593,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-03,0.001337,0.000814,0.005053,0.006452,0.025444,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-04,0.005785,-0.004185,-0.003372,-0.005978,-0.006292,-0.002073,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011750,0.000294,-0.013288,-0.048333,0.005127,-0.028092,-0.006509
2021-06-07,-0.005929,-0.031516,0.005537,0.007246,-0.022379,-0.003587,0.020669,-0.005232,0.008548,0.003556,...,-0.000093,0.009785,0.005353,0.009756,0.008807,0.003723,0.026011,0.003156,0.010131,0.017690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-15,0.001827,-0.000956,0.017757,0.011754,-0.011751,0.009499,0.011175,0.003938,0.014614,-0.000302,...,0.003435,0.005615,0.007850,0.015989,0.027507,0.002717,0.005303,0.003327,-0.005737,-0.009315
2022-08-16,-0.007787,-0.001244,0.007695,0.001014,-0.022951,-0.003148,-0.018511,0.004470,-0.015530,-0.008769,...,-0.005800,-0.026058,0.003138,-0.012003,0.001943,-0.002051,-0.023371,-0.012066,-0.004548,0.008091
2022-08-17,-0.001061,0.000383,0.001399,0.004974,0.021415,0.012771,0.001407,-0.016801,-0.007216,0.004576,...,0.008129,0.004426,0.014451,0.004050,-0.008834,0.000404,-0.024594,0.003403,0.001818,0.023644


In [3]:
rd2=rd1.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-02,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-03,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-04,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-07,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-12,12,35,31,34,8,5,11,20,6,10,...,7,28,14,13,18,21,27,45,44,2
2022-08-15,17,12,43,37,1,32,36,23,40,15,...,21,28,30,42,44,19,27,20,7,2
2022-08-16,19,31,42,32,7,28,8,40,11,17,...,21,4,39,14,35,30,6,13,25,43


In [4]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,2,4,1,1,4,3,4,5,1,3,...,2,5,1,4,1,2,2,5,4,4
2021-06-02,5,4,5,2,1,4,1,3,3,4,...,3,1,3,1,1,4,1,2,2,4
2021-06-03,2,2,4,4,5,4,4,1,4,3,...,2,5,4,3,2,1,1,4,3,3
2021-06-04,5,3,3,2,2,4,3,4,1,5,...,3,2,2,1,4,1,1,4,1,2
2021-06-07,1,1,4,4,1,2,5,1,4,3,...,3,4,3,4,4,3,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-12,2,4,4,4,1,1,2,3,1,2,...,1,4,2,2,2,3,3,5,5,1
2022-08-15,2,2,5,5,1,4,4,3,5,2,...,3,4,4,5,5,3,3,3,1,1
2022-08-16,3,4,5,4,1,4,1,5,2,2,...,3,1,5,2,4,4,1,2,3,5


#####Features
***

In [5]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-29','2022-08-22') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Functions
***

In [7]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [8]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [9]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-56] #only the days of one year until 3/01
      
      Y_train= y.iloc[:-56]

      print(Y_train.tail(1))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      for i in range(55,0,-1): #from 4/01 and later -->predictions
        X_validation=X.iloc[-(i+1):-i] 
        print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob

In [10]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [11]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

#####Gradient Boosting (baseline)
***

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1=predictions(baseline,assets,stock,29)
#---probabilities---#
prob1['date']=stock['ABBV'].iloc[-56:].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
[1 rows x 24 columns]
            (open2-open1)/open2  (high2-high1)/high2  (low2-low1)/low2  \
Date                                                                     
2022-07-26            -0.000171            -0.010027         -0.011227   

            (Adj Close2-Adj Close1)/Adj Close2      av_3      av_6     av_10  \
Date                                                                           
2022-07-26                           -0.015458  0.006397 -0.000089  0.002417   

               av_14  high1-low1/high1  close1-open1/close1  ...   TRIX_14  \
Date                                                         ...             
2022-07-26  0.003349           0.01627             0.006384  ...  0.247515   

             TRIX_10     WILLR    std_dev  dist_from_mean_10  \
Date                                                           
2022-07-26  0.281701 -0.109649  10.828185          10.091333   

            dist_from_mean_6   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-31,0.171173,0.114315,0.327186,0.517914,0.178463,0.221711,0.050690,0.179422,0.385111,0.165142,...,0.165475,0.625548,0.291956,0.147228,0.000039,0.205178,0.159494,0.129456,0.343956,0.393717
2022-06-01,0.275066,0.644415,0.033752,0.061297,0.245618,0.011947,0.003540,0.218599,0.127915,0.107800,...,0.183902,0.736033,0.098619,0.335825,0.097796,0.087252,0.315156,0.203019,0.175598,0.211551
2022-06-02,0.111710,0.241621,0.290744,0.062320,0.153215,0.122440,0.740724,0.055379,0.179075,0.088131,...,0.304765,0.328828,0.153388,0.252796,0.120022,0.073743,0.733110,0.089189,0.306983,0.237960
2022-06-03,0.273249,0.189530,0.184213,0.065446,0.216632,0.060221,0.148868,0.224804,0.192732,0.155122,...,0.116561,0.148194,0.289784,0.287223,0.245668,0.285725,0.177508,0.052887,0.006385,0.011907
2022-06-06,0.143621,0.213118,0.088947,0.511303,0.196102,0.319006,0.710693,0.093466,0.151698,0.046903,...,0.108136,0.747182,0.092560,0.194950,0.286380,0.128922,0.404784,0.195949,0.062050,0.538041
2022-06-07,0.314654,0.154988,0.149279,0.082955,0.462890,0.174037,0.267347,0.321003,0.089574,0.093807,...,0.091417,0.176109,0.393055,0.181586,0.144045,0.193150,0.002507,0.204645,0.293887,0.497823
2022-06-08,0.270125,0.044888,0.197806,0.151222,0.357324,0.171208,0.704029,0.031611,0.343986,0.272003,...,0.238474,0.005803,0.065442,0.325474,0.122062,0.362027,0.138160,0.313071,0.000068,0.140622
2022-06-09,0.210453,0.025077,0.122832,0.341532,0.179443,0.123942,0.401947,0.388394,0.135050,0.115904,...,0.193043,0.004077,0.359350,0.770833,0.274899,0.142310,0.034745,0.125167,0.178575,0.150851
2022-06-10,0.434579,0.184177,0.128849,0.006754,0.529051,0.243658,0.528785,0.240731,0.222663,0.344181,...,0.144203,0.120137,0.170170,0.554958,0.087594,0.094884,0.213882,0.052231,0.417760,0.219344


In [13]:
RPS1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-31,0.171173,0.114315,0.327186,0.517914,0.178463,0.221711,0.050690,0.179422,0.385111,0.165142,...,0.165475,0.625548,0.291956,0.147228,0.000039,0.205178,0.159494,0.129456,0.343956,0.393717
2022-06-01,0.275066,0.644415,0.033752,0.061297,0.245618,0.011947,0.003540,0.218599,0.127915,0.107800,...,0.183902,0.736033,0.098619,0.335825,0.097796,0.087252,0.315156,0.203019,0.175598,0.211551
2022-06-02,0.111710,0.241621,0.290744,0.062320,0.153215,0.122440,0.740724,0.055379,0.179075,0.088131,...,0.304765,0.328828,0.153388,0.252796,0.120022,0.073743,0.733110,0.089189,0.306983,0.237960
2022-06-03,0.273249,0.189530,0.184213,0.065446,0.216632,0.060221,0.148868,0.224804,0.192732,0.155122,...,0.116561,0.148194,0.289784,0.287223,0.245668,0.285725,0.177508,0.052887,0.006385,0.011907
2022-06-06,0.143621,0.213118,0.088947,0.511303,0.196102,0.319006,0.710693,0.093466,0.151698,0.046903,...,0.108136,0.747182,0.092560,0.194950,0.286380,0.128922,0.404784,0.195949,0.062050,0.538041
2022-06-07,0.314654,0.154988,0.149279,0.082955,0.462890,0.174037,0.267347,0.321003,0.089574,0.093807,...,0.091417,0.176109,0.393055,0.181586,0.144045,0.193150,0.002507,0.204645,0.293887,0.497823
2022-06-08,0.270125,0.044888,0.197806,0.151222,0.357324,0.171208,0.704029,0.031611,0.343986,0.272003,...,0.238474,0.005803,0.065442,0.325474,0.122062,0.362027,0.138160,0.313071,0.000068,0.140622
2022-06-09,0.210453,0.025077,0.122832,0.341532,0.179443,0.123942,0.401947,0.388394,0.135050,0.115904,...,0.193043,0.004077,0.359350,0.770833,0.274899,0.142310,0.034745,0.125167,0.178575,0.150851
2022-06-10,0.434579,0.184177,0.128849,0.006754,0.529051,0.243658,0.528785,0.240731,0.222663,0.344181,...,0.144203,0.120137,0.170170,0.554958,0.087594,0.094884,0.213882,0.052231,0.417760,0.219344


In [14]:
print(RPS1.mean(axis=1).mean())

0.2082085053205718


In [15]:
print(RPS1.mean(axis=1))

date
2022-05-31    0.211575
2022-06-01    0.220265
2022-06-02    0.221378
2022-06-03    0.174103
2022-06-06    0.221080
2022-06-07    0.203964
2022-06-08    0.207126
2022-06-09    0.185628
2022-06-10    0.228284
2022-06-13    0.220847
2022-06-14    0.218720
2022-06-15    0.200630
2022-06-16    0.257205
2022-06-17    0.176868
2022-06-21    0.221487
2022-06-22    0.217615
2022-06-23    0.213975
2022-06-24    0.219019
2022-06-27    0.203837
2022-06-28    0.203143
2022-06-29    0.228463
2022-06-30    0.234816
2022-07-01    0.202364
2022-07-05    0.225406
2022-07-06    0.189165
2022-07-07    0.186992
2022-07-08    0.184315
2022-07-11    0.208630
2022-07-12    0.194300
2022-07-13    0.217591
2022-07-14    0.234106
2022-07-15    0.185112
2022-07-18    0.186446
2022-07-19    0.219585
2022-07-20    0.206368
2022-07-21    0.183516
2022-07-22    0.223310
2022-07-25    0.198169
2022-07-26    0.200373
2022-07-27    0.194202
2022-07-28    0.153349
2022-07-29    0.199063
2022-08-01    0.219352
2022-0

In [22]:
#For 24 June
print(RPS1.iloc[16].mean())
#For 22 July
print(RPS1.iloc[35].mean())
#For 19 August
print(RPS1.iloc[55].mean())

0.21397481545759886
0.18351551948525546
0.22259881948243557


#####Gradient Boosting (with feature selection and tuning) 
***

In [23]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2=predictions(model1,assets,stock1,21)
#---probabilities---#
prob2['date']=stock['ABBV'].iloc[-56:].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
            dist_from_mean_10       PPO  
Date                                     
2022-08-16          -0.529731 -1.485078  
            (open2-open1)/open2  (low2-low1)/low2    av_3      av_6     av_14  \
Date                                                                            
2022-08-17             0.003271          0.000273  0.0076 -0.000854  0.004231   

            high1-low1/high1  close1-open1/close1       RSI       ADX  \
Date                                                                    
2022-08-17          0.034583              0.02454  0.560514  0.286754   

              ADX_14   TRIX_14   TRIX_10    WILLR   std_dev  \
Date                                                          
2022-08-17  0.123234  0.231429  0.149875 -0.46978  0.449784   

            dist_from_mean_10       PPO  
Date                                     
2022-08-17           0.154071 -1.661416  
DatetimeIndex(['2022-08-18'], dtype='da

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-31,0.085649,0.113621,0.252283,0.261155,0.215887,0.274625,0.261431,0.121296,0.275686,0.108564,...,0.074096,0.146404,0.135678,0.126808,0.213650,0.069991,0.130152,0.078852,0.251741,0.190326
2022-06-01,0.152654,0.250375,0.117104,0.071973,0.137036,0.070582,0.258169,0.115847,0.124812,0.131458,...,0.258108,0.306115,0.070393,0.239610,0.112253,0.279360,0.128435,0.135063,0.117624,0.263015
2022-06-02,0.194302,0.117553,0.254530,0.235866,0.227781,0.135434,0.196256,0.234841,0.124812,0.110580,...,0.253110,0.146508,0.111371,0.217439,0.090893,0.135691,0.238618,0.076677,0.113415,0.190326
2022-06-03,0.285898,0.119704,0.110184,0.073670,0.146229,0.099484,0.286702,0.234841,0.124812,0.108792,...,0.117713,0.134898,0.212018,0.088472,0.213650,0.105292,0.219752,0.077156,0.057437,0.154826
2022-06-06,0.194302,0.120181,0.110184,0.124439,0.146229,0.080761,0.196256,0.248673,0.066497,0.074596,...,0.071272,0.306115,0.070393,0.117304,0.213650,0.105292,0.223961,0.115052,0.117318,0.190326
2022-06-07,0.194302,0.120173,0.119503,0.227629,0.227781,0.127730,0.097115,0.238349,0.105125,0.108792,...,0.071272,0.134898,0.280841,0.129160,0.254082,0.105292,0.205378,0.123965,0.256013,0.184993
2022-06-08,0.152654,0.079665,0.110184,0.124615,0.227781,0.123996,0.196329,0.111804,0.105125,0.242882,...,0.121397,0.146508,0.111371,0.119794,0.133501,0.218116,0.109370,0.231187,0.057437,0.100878
2022-06-09,0.095626,0.236375,0.059103,0.124615,0.227781,0.238087,0.196256,0.254013,0.066497,0.108792,...,0.121397,0.131408,0.219322,0.225317,0.112253,0.105292,0.215417,0.115052,0.122150,0.101047
2022-06-10,0.194302,0.243929,0.254530,0.066556,0.227781,0.127591,0.196329,0.075485,0.105125,0.242964,...,0.121397,0.146508,0.113540,0.117304,0.133501,0.103451,0.215417,0.077214,0.272167,0.154826


In [25]:
print(RPS2.mean(axis=1).mean())

0.15876491164436196


In [26]:
print(RPS2.mean(axis=1))

date
2022-05-31    0.158533
2022-06-01    0.167305
2022-06-02    0.148257
2022-06-03    0.161680
2022-06-06    0.151514
2022-06-07    0.163489
2022-06-08    0.157646
2022-06-09    0.148832
2022-06-10    0.153081
2022-06-13    0.163104
2022-06-14    0.165891
2022-06-15    0.154128
2022-06-16    0.165495
2022-06-17    0.151935
2022-06-21    0.159645
2022-06-22    0.166034
2022-06-23    0.163687
2022-06-24    0.151808
2022-06-27    0.152578
2022-06-28    0.160460
2022-06-29    0.154936
2022-06-30    0.162279
2022-07-01    0.168064
2022-07-05    0.155853
2022-07-06    0.158981
2022-07-07    0.151725
2022-07-08    0.153505
2022-07-11    0.159074
2022-07-12    0.159565
2022-07-13    0.162550
2022-07-14    0.163002
2022-07-15    0.159854
2022-07-18    0.163404
2022-07-19    0.162567
2022-07-20    0.166916
2022-07-21    0.151380
2022-07-22    0.152373
2022-07-25    0.152443
2022-07-26    0.163940
2022-07-27    0.162191
2022-07-28    0.160169
2022-07-29    0.164851
2022-08-01    0.162622
2022-0

In [45]:
#For 24 June
print(RPS2.iloc[16].mean())
#For 22 July
print(RPS2.iloc[35].mean())
#For 19 August
print(RPS2.iloc[55].mean())

0.16368678876560389
0.15137985466693454
0.14967610130493142


In [28]:
#save the results 
RPS2.to_excel('Q2-Gradient_Boosting_next_day.xlsx')

####Gradient Boosting next month
***

#####Load Dataset 
***

In [29]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
#αλλαγή σε παραπάνω μήνες (+3)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-09-25')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο 
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-09-25')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd1= rd.pct_change()
rd1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,-0.007219,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002199,...,-0.004424,0.011185,-0.010431,0.004739,-0.016101,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-03,0.007271,0.003619,0.013996,-0.002891,-0.018919,0.004302,-0.014527,0.001645,-0.000593,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-04,0.001337,0.000814,0.005053,0.006452,0.025444,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-07,0.005785,-0.004185,-0.003372,-0.005978,-0.006293,-0.002072,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011750,0.000294,-0.013288,-0.048333,0.005127,-0.028091,-0.006509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,0.005798,0.004489,-0.001283,0.015859,-0.002176,0.009176,0.023861,-0.001119,0.005234,0.025309,...,0.012998,0.014756,0.000495,0.003876,0.015720,0.013178,0.007264,0.006969,0.004114,0.011503
2022-09-13,-0.026083,-0.028332,-0.030583,-0.052564,-0.061371,-0.039828,-0.070575,-0.029736,-0.038428,-0.043706,...,-0.039796,-0.027035,-0.011844,-0.009101,-0.053906,-0.032546,-0.059764,-0.033683,-0.034123,-0.023358
2022-09-14,0.007363,0.010959,-0.016216,-0.007717,0.002766,0.000864,0.013641,-0.013327,-0.024259,0.006115,...,-0.017852,0.027892,0.002397,0.011133,-0.004451,-0.008152,-0.015848,-0.001302,-0.000161,0.024546


In [30]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,0.014382,0.007921,0.004290,0.006533,-0.031250,0.008478,-0.002090,0.008721,0.000143,0.012986,...,0.008100,-0.008508,0.019087,0.011988,0.007984,0.011113,0.007335,0.005688,0.011905,0.002854
2021-06-02,0.007964,-0.000704,-0.002232,-0.001213,0.001522,-0.000279,0.022724,0.003895,-0.001760,0.009345,...,-0.001646,0.004291,-0.017405,0.001445,0.010081,0.011040,-0.005539,0.014799,0.003959,-0.001423
2021-06-03,0.004862,0.000470,0.001981,-0.010568,-0.009988,-0.014865,0.046927,0.010505,-0.016103,0.006291,...,-0.028993,0.008269,-0.015732,0.008367,0.002139,0.002199,-0.025472,0.004065,0.000620,-0.028495
2021-06-04,0.008814,0.004458,0.001020,0.010537,-0.013964,0.007039,0.005670,0.010208,0.014478,0.008375,...,0.002596,0.015343,0.010468,0.018026,-0.019206,0.003412,0.004367,0.001669,0.025224,-0.015643
2021-06-07,-0.003426,-0.003621,-0.016691,-0.017855,-0.017350,-0.030531,0.009422,0.007139,-0.009689,-0.020353,...,-0.032371,-0.007000,-0.031835,-0.016020,-0.019502,-0.004154,-0.021005,-0.014125,-0.004833,-0.004470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,5,3,2,2,1,3,1,4,1,5,...,3,1,5,4,3,4,3,2,4,2
2021-06-02,4,2,1,2,3,2,5,3,1,5,...,1,3,1,2,5,5,1,5,3,2
2021-06-03,5,4,4,2,2,2,5,5,1,5,...,1,5,2,5,4,4,1,4,4,1
2021-06-04,4,2,1,4,1,3,3,4,5,3,...,2,5,4,5,1,2,2,2,5,1
2021-06-07,4,4,2,1,2,1,5,5,3,1,...,1,3,1,2,1,4,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-08-11,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5
2022-08-12,4,4,1,2,4,3,5,2,1,4,...,1,5,4,5,3,2,1,3,3,5


In [32]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-03,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-04,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-07,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-08,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-09-13,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37
2022-09-14,34,36,7,17,29,26,40,10,4,32,...,5,44,28,37,20,15,8,24,25,43


#####Features 
All of them 
***

In [33]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Iούνιο 2022 (3 μήνες παραπάνω δεδομένα για να τα χρησιμοποιήσουμε για τα target values)->προβλεψη τον επομενο μηνα
  stock[i]=pdr.get_data_yahoo(i,'2021-01-01','2022-09-01') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]
stock

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-01-04  107.180000  107.349998  103.860001  105.410004   97.798584
 2021-01-05  105.410004  107.019997  104.629997  106.500000   98.809891
 2021-01-06  104.750000  107.190002  104.180000  105.580002   97.956322
 2021-01-07  106.110001  107.059998  105.570000  106.709999   99.004730
 2021-01-08  106.839996  107.529999  105.760002  107.269997   99.524284
 ...                ...         ...         ...         ...         ...
 2022-08-25  138.770004  139.559998  137.669998  139.330002  139.330002
 2022-08-26  138.889999  139.850006  136.250000  136.350006  136.350006
 2022-08-29  136.199997  136.720001  134.759995  135.710007  135.710007
 2022-08-30  135.139999  136.490005  134.889999  135.550003  135.550003
 2022-08-31  135.690002  136.039993  134.169998  134.460007  134.460007
 
 [419 rows x 5 columns],
 'AEP':                   Ope

In [34]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)

    ####CHANGE THE RANK TO NEXT MONTH --> 22 days later (working)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Functions
***

In [35]:
def RPS_calculation2(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [36]:
def create_historical2(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [37]:
def predictions2(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      #print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-60] #only the days of one year until 3/01--->CHANGE FROM -60 TO -100

      # print("this is the training",X_train.tail(1))

      Y_train= y.iloc[:-60]

      print("this is target training",Y_train.tail(1))

      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(59,0,-1): #from 4/01 and later -->predictions--->CHANGE FROM (59,0,-1) TO (99,39,-1)
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        print('this is for validation',Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      last=X_validation.index
      print('this is last for prediction',last)
        #THAT WAS FOR LAST DAY -> NOT NECESSARY FOR THIS METHOD
      # X_validation=X.tail(1) 
      # print('this is last for prediction',X_validation)
      # Y_validation= y.tail(1)
      # print(Y_validation.index) 
      # probability= model.predict_proba(X_validation)
      # print(probability[0])
      # tmp.append(probability[0])
      prob[j]=tmp

      
  return prob ,first , last 

In [38]:
def RPS_data2(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]
      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [39]:
def create_RPS2(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

#####Gradient Boosting (baseline)
***

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
baseline2= GradientBoostingClassifier()
#---predictions---#
prob3, first , last = predictions2(baseline2,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob3['date']=stock['ABBV'].loc[mask].index
prob3_final=pd.DataFrame.from_dict(prob3)
prob3_final.set_index('date')
RPS3= RPS_data2(prob3_final,rd)
RPS3= create_RPS2(prob3_final,RPS3)
RPS3
# print(RPS3.mean(axis=1).mean())

this is target training Date
2022-05-19    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-24'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-25'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-26'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
thi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18:

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.180511,0.190229,0.110498,0.114742,0.212568,0.122122,0.293319,0.325328,0.186772,0.258140,...,0.017314,0.134221,0.272178,0.210365,0.112838,0.185626,0.147405,0.030261,0.197510,0.360209
2022-05-23,0.081676,0.235163,0.175738,0.085035,0.431732,0.033706,0.392256,0.415823,0.572137,0.155629,...,0.366903,0.268850,0.087014,0.131009,0.003509,0.468114,0.659099,0.137042,0.458033,0.189511
2022-05-24,0.250995,0.199422,0.176219,0.250740,0.154799,0.313773,0.029818,0.305798,0.103263,0.238644,...,0.186947,0.066304,0.123587,0.041279,0.352655,0.018913,0.419476,0.192707,0.004561,0.078530
2022-05-25,0.232794,0.210749,0.207138,0.014785,0.145610,0.151610,0.308390,0.174844,0.032189,0.144084,...,0.355126,0.035091,0.101954,0.378606,0.258015,0.185675,0.333588,0.090294,0.569646,0.193389
2022-05-26,0.134700,0.120180,0.358826,0.142184,0.023853,0.047755,0.396518,0.270666,0.121772,0.458424,...,0.183708,0.229643,0.130956,0.178743,0.136013,0.033382,0.161731,0.229604,0.157511,0.354195
2022-05-27,0.219919,0.258300,0.130360,0.057589,0.008765,0.280715,0.029889,0.030610,0.143301,0.196041,...,0.027048,0.607459,0.267472,0.003917,0.226369,0.119062,0.554088,0.092912,0.172293,0.320225
2022-05-31,0.100444,0.003486,0.081289,0.118068,0.395627,0.502772,0.669735,0.165379,0.098639,0.031396,...,0.161567,0.102019,0.023600,0.305612,0.429697,0.162323,0.102601,0.149282,0.011991,0.495890
2022-06-01,0.109338,0.253580,0.047353,0.144807,0.102871,0.060334,0.288133,0.265372,0.035545,0.070224,...,0.297258,0.424973,0.375135,0.126540,0.037581,0.226759,0.133501,0.187932,0.124893,0.014469
2022-06-02,0.336618,0.245034,0.203693,0.066350,0.087471,0.187835,0.315443,0.195432,0.449553,0.194571,...,0.073906,0.005550,0.073065,0.265101,0.050257,0.272405,0.627200,0.151266,0.159949,0.195145


In [41]:
print(RPS3.mean(axis=1).mean())

0.20349831654974215


In [42]:
print(RPS3.mean(axis=1))

date
2022-05-20    0.162425
2022-05-23    0.222742
2022-05-24    0.156769
2022-05-25    0.191307
2022-05-26    0.196854
2022-05-27    0.214597
2022-05-31    0.215002
2022-06-01    0.206303
2022-06-02    0.167340
2022-06-03    0.221978
2022-06-06    0.198169
2022-06-07    0.154891
2022-06-08    0.256933
2022-06-09    0.200861
2022-06-10    0.215438
2022-06-13    0.270044
2022-06-14    0.232872
2022-06-15    0.211579
2022-06-16    0.235093
2022-06-17    0.217618
2022-06-21    0.171907
2022-06-22    0.186044
2022-06-23    0.155118
2022-06-24    0.224852
2022-06-27    0.207450
2022-06-28    0.219545
2022-06-29    0.235717
2022-06-30    0.216501
2022-07-01    0.198536
2022-07-05    0.211348
2022-07-06    0.200892
2022-07-07    0.201510
2022-07-08    0.189239
2022-07-11    0.177125
2022-07-12    0.212464
2022-07-13    0.194860
2022-07-14    0.245334
2022-07-15    0.212760
2022-07-18    0.225173
2022-07-19    0.188018
2022-07-20    0.199340
2022-07-21    0.211792
2022-07-22    0.239002
2022-0

In [53]:
#For 24 June
print(RPS3.iloc[1].mean())
#For 22 July
print(RPS3.iloc[20].mean())
#For 19 August
print(RPS3.iloc[40].mean())

0.22274249920926303
0.17190746360974943
0.19933984809517066


#####Gradient Boosting (with feature selection and tuning)
***

In [54]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [55]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob4 ,first,last =predictions2(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob4['date']=stock['ABBV'].loc[mask].index
prob4_final=pd.DataFrame.from_dict(prob4)
prob4_final.set_index('date')
RPS4= RPS_data2(prob4_final,rd)
RPS4= create_RPS2(prob4_final,RPS4)
RPS4
# print(RPS4.mean(axis=1).mean())

this is target training Date
2022-05-19    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-24'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-25'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-26'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
thi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18:

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-16  142.550003

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.086145,0.236039,0.281111,0.079076,0.134429,0.133839,0.259916,0.239800,0.104592,0.234667,...,0.113071,0.134347,0.134122,0.219990,0.129247,0.100421,0.201831,0.073892,0.241447,0.246960
2022-05-23,0.145011,0.249624,0.119054,0.076066,0.145096,0.102031,0.152589,0.251739,0.277506,0.074803,...,0.252518,0.294415,0.109997,0.082716,0.129247,0.276376,0.243076,0.121040,0.273102,0.150100
2022-05-24,0.107463,0.236039,0.062254,0.109963,0.118600,0.133806,0.197806,0.240753,0.121745,0.234667,...,0.113071,0.152680,0.069311,0.087905,0.204489,0.220252,0.242000,0.233747,0.100805,0.186415
2022-05-25,0.097996,0.236039,0.252215,0.131291,0.134429,0.073878,0.197806,0.109573,0.106020,0.074803,...,0.252518,0.153137,0.110993,0.137819,0.204489,0.220252,0.115621,0.115845,0.273102,0.199535
2022-05-26,0.097751,0.111840,0.121066,0.128179,0.208616,0.268256,0.259916,0.076573,0.124113,0.240089,...,0.113071,0.129581,0.130570,0.214841,0.109707,0.220252,0.201831,0.123478,0.251291,0.259002
2022-05-27,0.087074,0.236393,0.243955,0.076102,0.138506,0.133806,0.198950,0.076153,0.106020,0.132763,...,0.070645,0.153137,0.222515,0.107977,0.208901,0.071725,0.243076,0.114320,0.255472,0.246960
2022-05-31,0.151064,0.112109,0.059712,0.128179,0.219152,0.268256,0.286227,0.121464,0.065509,0.075753,...,0.121492,0.280427,0.070141,0.257366,0.137431,0.142178,0.122840,0.074940,0.055666,0.199535
2022-06-01,0.095150,0.249062,0.122567,0.123858,0.115191,0.133839,0.287130,0.123737,0.065509,0.075753,...,0.247225,0.294415,0.277435,0.087905,0.256496,0.276376,0.143155,0.123478,0.118747,0.246960
2022-06-02,0.282893,0.236393,0.107561,0.076102,0.145096,0.133839,0.174752,0.123748,0.271747,0.240089,...,0.117300,0.153137,0.216238,0.214841,0.117241,0.220252,0.205341,0.074796,0.251291,0.246960


In [56]:
print(RPS4.mean(axis=1).mean())

0.1591116140188601


In [57]:
print(RPS4.mean(axis=1))

date
2022-05-20    0.160138
2022-05-23    0.168291
2022-05-24    0.151124
2022-05-25    0.153666
2022-05-26    0.156823
2022-05-27    0.154580
2022-05-31    0.161481
2022-06-01    0.167249
2022-06-02    0.155475
2022-06-03    0.161244
2022-06-06    0.153217
2022-06-07    0.151331
2022-06-08    0.161369
2022-06-09    0.158814
2022-06-10    0.161836
2022-06-13    0.166641
2022-06-14    0.163206
2022-06-15    0.164182
2022-06-16    0.164926
2022-06-17    0.167430
2022-06-21    0.150734
2022-06-22    0.153446
2022-06-23    0.152110
2022-06-24    0.166522
2022-06-27    0.162531
2022-06-28    0.161814
2022-06-29    0.163908
2022-06-30    0.162793
2022-07-01    0.164062
2022-07-05    0.162599
2022-07-06    0.150217
2022-07-07    0.156528
2022-07-08    0.149706
2022-07-11    0.165422
2022-07-12    0.161347
2022-07-13    0.165226
2022-07-14    0.167798
2022-07-15    0.162692
2022-07-18    0.149062
2022-07-19    0.155580
2022-07-20    0.149673
2022-07-21    0.147473
2022-07-22    0.161841
2022-0

In [58]:
#For 24 June
print(RPS4.iloc[1].mean())
#For 22 July
print(RPS4.iloc[20].mean())
#For 19 August
print(RPS4.iloc[40].mean())

0.1682913866074132
0.15073355361603993
0.14967272617521304


In [59]:
#save results
RPS4.to_excel('Q2-Gradient Boosting_next_month_prediction.xlsx')